# M03 Homework

- Name: Sam Remmey
- Net ID: sqr8ap
- URL of this file in GitHub: https://github.com/sqr8ap/DS5001-2025-01-R/blob/m03/lessons/M03_LanguageModels/M03_HW.ipynb

Note: I parsed the text manually before realizing that the textimporter file contains a method for doing that

In [4]:
import numpy as np
import pandas as pd
import configparser
config = configparser.ConfigParser()

config.read("../../../env.ini")
data_home = config['DEFAULT']['data_home']
output_dir = config['DEFAULT']['output_dir']

!ls -l {data_home}

total 4336
-rw-r--r--@ 1 Samantha  staff  1752309 Jan 27 17:42 austen-persuasion.csv
drwxr-xr-x@ 8 Samantha  staff      256 Jan 16 08:52 gutenberg
-rw-r--r--@ 1 Samantha  staff   465627 Jan 20 15:27 pg42324.txt


In [23]:
import sys
import os

module_path = "/Users/Samantha/Desktop/MSDS/DS5001/repo/lessons/lib"

if module_path not in sys.path:
    sys.path.append(module_path)

import textimporter

## Parse the text to generate TOKENS and VOCAB tables.

In [167]:
OHCO = ['chap_num', 'para_num', 'sent_num', 'token_num']

In [26]:
text_file = f"{data_home}/pg42324.txt"

In [28]:
LINES = pd.DataFrame(open(text_file, 'r', encoding='utf-8-sig').readlines(), columns=['line_str'])
LINES.index.name = 'line_num'
LINES.line_str = LINES.line_str.str.replace(r'\n+', ' ', regex=True).str.strip()

In [48]:
clip_pats = [
    r"^PREFACE\b",  # Match a line that starts with "PREFACE"
    r"^THE END\.$"  # Match a line that contains exactly "THE END."
]

pat_a = LINES.line_str.str.match(clip_pats[0])
pat_b = LINES.line_str.str.match(clip_pats[1])

In [66]:
line_a = LINES.loc[pat_a].index[0] # first line of story ('PREFACE')
line_b = LINES.loc[pat_b].index[0] + 1 # last line of story ('THE END.')

In [68]:
LINES = LINES.loc[line_a : line_b]
print(LINES.head(2))
print(LINES.tail(2))

          line_str
line_num          
271       PREFACE.
272               
          line_str
line_num          
7655      THE END.
7656              


In [74]:
## Chunk by chapter

chap_pat = r"^\s*(?:PREFACE|chapter\s+[IVXLCDM]+|letter\s+[IVXLCDM]+)"

chap_lines = LINES.line_str.str.match(chap_pat, case=False)

LINES.loc[chap_lines].head(3)

,line_str
line_num,
271,PREFACE.
343,LETTER I.
467,LETTER II.


In [78]:
LINES.loc[chap_lines, 'chap_num'] = [i+1 for i in range(LINES.loc[chap_lines].shape[0])]

In [80]:
LINES.loc[chap_lines].head()

,line_str,chap_num
line_num,,
271,PREFACE.,1.0
343,LETTER I.,2.0
467,LETTER II.,3.0
594,LETTER III.,4.0
636,LETTER IV.,5.0


In [85]:
LINES.chap_num = LINES.chap_num.ffill()

/var/folders/13/9xyr4p8x2kq9msv1m8ysrjq00000gn/T/ipykernel_44039/3599576442.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LINES.chap_num = LINES.chap_num.ffill()


In [99]:
LINES.sample(3)

,line_str,chap_num
line_num,,
2515,"discover what I knew of the murderer, and caus...",12
6810,"accuse, and for whose seizure and punishment I...",28
1867,view. I did not dare return to the apartment w...,10


In [95]:
LINES = LINES.loc[~chap_lines] # Remove chapter heading lines; their work is done
LINES.chap_num = LINES.chap_num.astype('int') # Convert chap_num from float to int

In [97]:
LINES.sample(3)

,line_str,chap_num
line_num,,
3101,"exercise and by change of place, some relief f...",14
4555,"""Cursed, cursed creator! Why did I live? Why, ...",21
2252,always been my favourite companion in the ramb...,11


In [105]:
OHCO[:1]

['chap_num']

In [107]:
# Make big string for each chapter
CHAPS = LINES.groupby(OHCO[:1])\
    .line_str.apply(lambda x: '\n'.join(x))\
    .to_frame('chap_str')

In [109]:
CHAPS.head(3)

,chap_str
chap_num,
1,\n\nThe event on which this fiction is founded...
2,"\n\n_To Mrs. Saville, England._\n\nSt. Petersb..."
3,"\n\n_To Mrs. Saville, England._\n\nArchangel, ..."


In [111]:
CHAPS['chap_str'] = CHAPS.chap_str.str.strip()

In [113]:
## Split chapters into paragraphs

para_pat = r'\n\n+'

In [115]:
PARAS = CHAPS['chap_str'].str.split(para_pat, expand=True).stack()\
    .to_frame('para_str').sort_index()
PARAS.index.names = OHCO[:2]

In [117]:
PARAS['para_str'] = PARAS['para_str'].str.replace(r'\n', ' ', regex=True)
PARAS['para_str'] = PARAS['para_str'].str.strip()
PARAS = PARAS[~PARAS['para_str'].str.match(r'^\s*$')] # Remove empty paragraphs

PARAS.head(3)

para_str
chap_num para_num                                                   
1        0         The event on which this fiction is founded, ha...
         1         I have thus endeavoured to preserve the truth ...
         2         The circumstance on which my story rests was s...

In [119]:
## Split paragraphs into sentences

sent_pat = r'[.?!;:]+'
SENTS = PARAS['para_str'].str.split(sent_pat, expand=True).stack()\
    .to_frame('sent_str')
SENTS.index.names = OHCO[:3]

In [121]:
SENTS = SENTS[~SENTS['sent_str'].str.match(r'^\s*$')] # Remove empty paragraphs
SENTS.sent_str = SENTS.sent_str.str.strip() # CRUCIAL TO REMOVE BLANK TOKENS

In [123]:
SENTS.sample(3)

,,,sent_str
chap_num,para_num,sent_num,
22,15,0,"""I consent to your demand, on your solemn oath..."
25,8,4,I left Switzerland with you
7,13,8,and entertained the greatest disdain for a wou...


In [125]:
## Split sentences into tokens

token_pat = r"[\s',-]+"
TOKENS = SENTS['sent_str'].str.split(token_pat, expand=True).stack()\
    .to_frame('token_str')

In [127]:
TOKENS.index.names = OHCO[:4]

In [131]:
TOKENS.sample(3)

,,,,token_str
chap_num,para_num,sent_num,token_num,
25,34,3,8,the
22,18,13,2,become
14,10,2,5,cloud


In [133]:
## Generate VOCAB table

TOKENS['term_str'] = TOKENS.token_str.replace(r'[\W_]+', '', regex=True).str.lower()
VOCAB = TOKENS.term_str.value_counts().to_frame('n').reset_index().rename(columns={'index':'term_str'})
VOCAB.index.name = 'term_id'

In [141]:
VOCAB['length'] = VOCAB['term_str'].str.len()
VOCAB = VOCAB.set_index('term_str')

In [143]:
VOCAB.sample(3)

,n,length
term_str,,
catastrophe,3,11
chimney,2,7
semblance,1,9


## Create a list of sentences from the TOKENS table and a list of terms from the VOCAB table.

#### I don't understand what this is asking for?

In [153]:
TOKENS.head()

token_str term_str
chap_num para_num sent_num token_num                   
1        0        0        0               The      the
                           1             event    event
                           2                on       on
                           3             which    which
                           4              this     this

In [155]:
VOCAB.head()

,n,length
term_str,,
the,4250,3
and,2993,3
i,2859,1
of,2684,2
to,2119,2


In [157]:
def token_to_padded(token, grouper=['sent_num'], term_str='term_str'):
    ohco = token.index.names # We preserve these since they get lost in the shuffle
    padded = token.groupby(grouper)\
        .apply(lambda x: '<s> ' + ' '.join(x[term_str]) + ' </s>')\
        .apply(lambda x: pd.Series(x.split()))\
        .stack().to_frame('term_str')
    padded.index.names = ohco
    return padded

In [171]:
PADDED = token_to_padded(TOKENS, grouper=OHCO[:3], term_str='term_str')

In [179]:
PADDED.head()

term_str
chap_num para_num sent_num token_num         
1        0        0        0              <s>
                           1              the
                           2            event
                           3               on
                           4            which

## Generate ngram type tables and models, going up to the trigram level.

In [188]:
ngrams = 3
widx = [f"w{i}" for i in range(ngrams)]

In [184]:
def padded_to_ngrams(padded, grouper=['sent_num'], n=2):
    
    ohco = padded.index.names
    ngrams = padded.groupby(grouper)\
        .apply(lambda x: pd.concat([x.shift(0-i) for i in range(n)], axis=1))\
        .reset_index(drop=True)
    ngrams.index = padded.index
    ngrams.columns = widx

    # ngrams = pd.concat([padded.shift(0-i) for i in range(n)], axis=1)
    # ngrams.index.name = 'ngram_num'
    # ngrams.columns = widx
    # ngrams = ngrams.fillna('<EOF>')
    
    return ngrams

In [190]:
NGRAMS = padded_to_ngrams(PADDED, OHCO[:3], n=ngrams)

In [208]:
NGRAMS.head()

w0     w1       w2
chap_num para_num sent_num token_num                       
1        0        0        0            <s>    the    event
                           1            the  event       on
                           2          event     on    which
                           3             on  which     this
                           4          which   this  fiction

In [194]:
def ngrams_to_models(ngrams):
    global widx
    n = len(ngrams.columns)
    model = [None for i in range(n)]
    for i in range(n):
        if i == 0:
            model[i] = ngrams.value_counts('w0').to_frame('n')
            model[i]['p'] = model[i].n / model[i].n.sum()
            model[i]['i'] = np.log2(1/model[i].p)
        else:
            model[i] = ngrams.value_counts(widx[:i+1]).to_frame('n')    
            model[i]['cp'] = model[i].n / model[i-1].n
            model[i]['i'] = np.log2(1/model[i].cp)
        model[i] = model[i].sort_index()
    return model

In [196]:
M = ngrams_to_models(NGRAMS)

In [206]:
M[2].head()

n   cp    i
w0   w1  w2                  
11th 17  </s>     1  1.0  0.0
     the passage  1  1.0  0.0
12th 17  </s>     1  1.0  0.0
13th 17  </s>     1  1.0  0.0
1816 in  the      1  1.0  0.0

## Questions:

#### 1. List six words that precede the word "monster," excluding stop words (and sentence boundary markers). Stop words include 'a', 'an', 'the', 'this', 'that', etc. Hint: use the df.query() method.

In [224]:
stop_words = {'a', 'an', 'the', 'this', 'that', 'is', 'of', 'in', 'to', 'and', 'on', 'for', 'with', 'as', 'by'}

# Extract preceding words from bigrams
preceding_words = NGRAMS.query('w2 == "monster"')['w1']

filtered_words = [word for word in preceding_words if word.lower() not in stop_words]

In [226]:
filtered_words

['miserable', 'abhorred', 'detestable', 'hideous', 'hellish', 'gigantic']

#### 2. List the following sentences in ascending order of bigram perplexity according to the language model generated from the text:

- The monster is on the ice.
- Flowers are happy things.
- I have never seen the aurora borealis.
- He never knew the love of a family.

In [248]:
test_sents = ['the monster is on the ice',
             'flowers are happy things',
             'i have never seen the aurora borealis',
             'he never knew the love of a family']
test_sents = pd.DataFrame(test_sents)
test_sents.index.name = 'sent_num'
test_sents.columns = ['sent_str']

In [250]:
test_sents

,sent_str
sent_num,
0,the monster is on the ice
1,flowers are happy things
2,i have never seen the aurora borealis
3,he never knew the love of a family


In [252]:
def test_model(model, ngrams, sents):
    
    global widx
    
    assert len(model) == len(ngrams.columns)
    
    n = len(model)
    ohco = ngrams.index.names
    
    R = []
    for i in range(n):
        T = ngrams.merge(M[i], on=widx[:i+1], how='left')
        T.index = ngrams.index
        T = T.reset_index().set_index(ohco + widx).i #.to_frame(f"i{i}")
        
        # This how we handle unseen combos
        T[T.isna()] = T.max()
        R.append(T.to_frame(f"i{i}"))
                
    return pd.concat(R, axis=1)

In [258]:
R = test_model(M, NGRAMS, test_sents)

In [260]:
def compute_perplexity(results, test_sents, n=3):
    for i in range(n):
        test_sents[f"pp{i}"] = np.exp2(results.groupby('sent_num')[f"i{i}"].mean())
    return test_sents

In [262]:
PP = compute_perplexity(R, test_sents)

In [264]:
PP

,sent_str,pp0,pp1,pp2
sent_num,,,,
0,the monster is on the ice,440.981155,46.227546,6.924685
1,flowers are happy things,450.465890,45.964740,6.555687
2,i have never seen the aurora borealis,450.926586,45.953661,6.451554
3,he never knew the love of a family,442.846258,46.221218,6.643836


In ascending order:

- i have never seen the aurora borealis
- flowers are happy things
- he never knew the love of a family
- the monster is on the ice

#### 3. Using the bigram model represented as a matrix, explore the relationship between bigram pairs using the following lists. Hint: use the .unstack() method on the feature n and then use .loc[] to select the first list from the index, and the second list from the columns.

- ['he','she'] to select the indices.
- ['said','heard'] to select the columns

In [310]:
M[1].n.unstack().loc[['he','she'],['said','heard']]

w1,said,heard
w0,,
he,21.0,5.0
she,3.0,3.0


It looks like 'he said' is a common bigram pair. 

#### 4. Generate 20 sentences using the generate_text() function. Display the results.

In [278]:
def generate_text(M, n=250):
    
    if len(M) < 3:
        raise ValueError("Must have trigram model generated.")
    
    # Start list of words
    first_word = M[1].loc['<s>'].sample(weights='cp').index[0]
    
    words = ['<s>', first_word]
    
    for i in range(n):
        
        bg = tuple(words[-2:])

        # Try trigram model
        try:
            next_word = M[2].loc[bg].sample(weights='cp').index[0]

        # If not found in model, back off ...
        except KeyError as e1:
            try:
                # Get the last word in the bigram
                ug = bg[1]
                next_word = M[1].loc[ug].sample(weights='cp').index[0]
            
            except KeyError as e2:
                next_word = M[0].sample(weights='p').index[0]
                
        words.append(next_word)
    
    
    text = ' '.join(words[2:])
    print('\n\n'.join([str(i+1) + ' ' + line.replace('<s>','')\
        .strip().upper() for i, line in enumerate(text.split('</s>'))]))

In [292]:
generate_text(M, 300)

1 RESOLVED NOT TO JOIN HIM

2 JUSTINE WERE ASSASSINATED AND THE YOUTH JOINED HER WHO SUFFERED THROUGH ME

3 

4 EVERY CONVERSATION OF THE UNEASINESS OF THIS OPINION

5 TREMBLE

6 LABOUR WAS ALREADY ONE IN MY MEMORY AND I WILL RELATE HER HISTORY THEREFORE IN THIS MANNER

7 BELONGED TO HER ON THE SHOULDER AND I WAS BENEVOLENT

8 

9 I HAD WALKED WITH CAUTION

10 THIS SACRIFICE

11 UNSATISFACTORY

12 GREECE SHAKSPEARE IN THE HALL OF OUR COUNTRY HAVE PRODUCED IT YET THE TALE WAS QUICKLY RESTORED BY THE COFFIN HIS FACE WAS CONCEALED BY LONG LOCKS OF RAGGED HAIR

13 CREATURES WERE MISERABLE IT WAS IMPOSSIBLE TO COMMUNICATE TO YOU THE AGONY OF MY HEART SINK WITHIN ME DIED IN HER HIDING PLACES TO SEEK FOR KNOWLEDGE AND WISDOM AS I SAID LITTLE

14 FROM THE OUT HOUSE WHERE TO HIS SISTER ESPECIALLY WHEN HE SPOKE I READ

15 SPIRITS BECAME UNEQUAL

16 THAT I WAS AT ONCE

17 LITTLE CHANCE OF MEETING

18 THEY WERE NOT THE WITNESS OF HIS CHILDREN I COULD GIVE ME ON ALL THAT AFFECTION WHICH A PRISON CA

#### 5.

In [333]:
M[1]['p'] = M[1].n /  M[1].n.sum()
M[2]['p'] = M[2].n /  M[2].n.sum()

In [364]:
## Unigram model

H = sum(M[0].p * np.log2(1 / M[0].p))
Hmax = np.log2(len(M[0].index)**1)
R = 1 - (H / Hmax)
R

0.3089912730771178

In [368]:
## Bigram model

H = sum(M[1].p * np.log2(1 / M[1].p))
Hmax = np.log2(len(M[1].index)**2)
R = 1 - (H / Hmax)
R

0.5388229906327178

In [370]:
## Trigram model

H = sum(M[2].p * np.log2(1 / M[2].p))
Hmax = np.log2(len(M[2].index)**3)
R = 1 - (H / Hmax)
R

0.6705947570115205

R increases as the choice of n-gram increases in length. 